# Выбор локации для скважины

Для добывающей компании «ГлавРосГосНефть» нужно решить, где бурить новую скважину.

Шаги для выбора локации обычно такие:
* В избранном регионе собирают характеристики для скважин: качество нефти и объём её запасов;
* Строят модель для предсказания объёма запасов в новых скважинах;
* Выбирают скважины с самыми высокими оценками значений;
* Определяют регион с максимальной суммарной прибылью отобранных скважин.

Предоставлены пробы нефти в трёх регионах. Характеристики для каждой скважины в регионе уже известны. Необходимо построить модель для определения региона, где добыча принесёт наибольшую прибыль. Проанализировать возможную прибыль и риски техникой Bootstrap.

Принятые допущения:
* Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
* При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.
* Бюджет на разработку скважин в регионе — 10 млрд рублей.
* При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
* После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

## Загрузка данных

Перед началолом работы разрузим все необходимые библиотеки.

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, confusion_matrix, plot_confusion_matrix, recall_score, mean_absolute_error
from sklearn.metrics import roc_curve, roc_auc_score, r2_score, precision_score, f1_score, precision_recall_curve
from sklearn.utils import shuffle

import matplotlib.pyplot as plt
from IPython.display import display
import seaborn as sns
import numpy as np
from sklearn.metrics import accuracy_score
import warnings
import re
from sklearn.preprocessing import OrdinalEncoder, StandardScaler

warnings.simplefilter(action='ignore', category=FutureWarning)

Данные геологоразведки трёх регионов находятся в файлах, загрузим их и выведем несколько строк для наглядности.

In [2]:
df_1 = pd.read_csv('C:/Users/valentina.tikhova/Downloads/yandex_ds/course8_Машинное обучение в бизнесе/geo_data_0.csv')
df_2 = pd.read_csv('C:/Users/valentina.tikhova/Downloads/yandex_ds/course8_Машинное обучение в бизнесе/geo_data_1.csv')
df_3 = pd.read_csv('C:/Users/valentina.tikhova/Downloads/yandex_ds/course8_Машинное обучение в бизнесе/geo_data_2.csv')

for idx, item in enumerate((df_1, df_2, df_3)):
    print('REGION', idx + 1)
    display(item.sample(3))

REGION 1


,id,f0,f1,f2,product
52973,RM3lO,-0.599243,0.721781,5.179657,60.766206
28540,7EsfC,0.028787,0.131433,-2.377188,51.323266
72045,MM8yj,0.283694,0.959426,3.057990,55.727858


REGION 2


,id,f0,f1,f2,product
6264,KFLP0,5.265481,1.484071,-0.001513,0.000000
88634,SwW1M,-9.901363,-0.552535,0.005803,3.179103
4340,xzrI7,0.075398,-10.758952,2.003220,57.085625


REGION 3


,id,f0,f1,f2,product
11195,rMR39,-1.582440,0.167386,2.511576,95.267873
2380,PD5BN,-2.522486,1.954481,4.598875,142.075504
29681,qAEJb,2.150432,-0.420461,5.616689,164.490737


В нашем распоряжении 100k записей, каждый объект в наборе данных — информация о скважине каждого региона, а именно:
- `id` — уникальный идентификатор скважины
- `f0, f1, f2` — три признака точек
- `product` — объём запасов в скважине (тыс. баррелей).


## EDA

Посмотрим информацию по выборке (и количественные и качественные переменные) методом describe(). Для наглядности выведем все в одной таблице.

In [3]:
pd.concat((i.describe(include = 'all').round(3) for i in [df_1, df_2, df_3]), keys=['REGION {}'.format(i) for i in range(1, 4)], axis=1).fillna('-')

REGION 1                                         REGION 2            \
             id        f0        f1        f2   product       id        f0   
count    100000  100000.0  100000.0  100000.0  100000.0   100000  100000.0   
unique    99990         -         -         -         -    99996         -   
top       fiKDv         -         -         -         -    wt4Uk         -   
freq          2         -         -         -         -        2         -   
mean          -       0.5      0.25     2.503      92.5        -     1.141   
std           -     0.872     0.504     3.248    44.289        -     8.966   
min           -    -1.409    -0.848   -12.088       0.0        -    -31.61   
25%           -    -0.073    -0.201     0.288    56.498        -    -6.299   
50%           -     0.502      0.25     2.516     91.85        -     1.153   
75%           -     1.074     0.701     4.715   128.564        -     8.621   
max           -     2.362     1.344    16.004   185.364        -    29.422   

                                     REGION 3                                \
              f1        f2   product       id        f0        f1        f2   
count   100000.0  100000.0  100000.0   100000  100000.0  100000.0  100000.0   
unique         -         -         -    99996         -         -         -   
top            -         -         -    VF7Jo         -         -         -   
freq           -         -         -        2         -         -         -   
mean      -4.797     2.495    68.825        -     0.002    -0.002     2.495   
std         5.12     1.704    45.944        -     1.732      1.73     3.473   
min      -26.359    -0.018       0.0        -     -8.76    -7.084    -11.97   
25%       -8.268       1.0    26.953        -    -1.162    -1.175      0.13   
50%       -4.813     2.011    57.086        -     0.009    -0.009     2.484   
75%       -1.333       4.0   107.813        -     1.159     1.164     4.859   
max       18.734      5.02   137.945        -     7.238     7.845    16.739   

                  
         product  
count   100000.0  
unique         -  
top            -  
freq           -  
mean        95.0  
std        44.75  
min          0.0  
25%        59.45  
50%       94.926  
75%      130.595  
max       190.03

В данных нет пропусков, есть одно категорийное поле в виде идентификатора скважины `id`, некоторые скважины имеют несколько записей в дата-сете (10 дублей в первом регионе и 4 повтора в остальных). Это небольшая погрешность (0,01%), она не будет влиять на конечный результат.
Признаки точек могут быть как положительные, так и отрицательные. По некоторым скважинам есть нулевой объем запасов.

In [4]:
df = pd.concat([df_1, df_2, df_3], keys=['reg_{}'.format(i) for i in range(1, 4)], axis = 0, names=['region', 'reg_id']).reset_index()
df.sample(3)

,region,reg_id,id,f0,f1,f2,product
251800,reg_3,51800,ju5Xk,-2.253866,0.637092,5.390571,162.733722
74412,reg_1,74412,HiHbz,0.024262,0.000391,-1.250224,57.278073
59455,reg_1,59455,z06a7,0.143753,-0.013447,-3.523956,34.598073


Посмотрим, какие параметры влиют больше всего на факт оттока из банка с помощью функции `corr()`. 

In [ ]:
df.corr().style.background_gradient(cmap='coolwarm', 
                                    axis=None, 
                                    vmin=df.corr().min().min(), 
                                    vmax=df.corr().max().max()).set_precision(3)

На факт оттока клиента больше всего влияет его возраст, а также баланс счета. Интересно, что чем активнее клиент, тем выше вероятность его ухода. Можно объяснить это заинтересованностью клиента в выгодных условиях банковских услуг. Вероятно, чем больше клиент использует банковские услуги, тем больше сравнивает их уровень у конкурентов.

Все же сильных взаимосвязей между признаками нет, ничего не будет менять будущей модели.

Чтобы составить начальный портрет клиентов, ушедших в отток, и лояльных клиентов, которые продолжаются пользоваться услугами банка, посмотрим всю ту же информацию `describe()` по целевому признаку `exited`:

In [ ]:
df.query("exited == 1").describe(percentiles=[.50, .9], include = 'all').round(2).fillna('-')

**Портрет пользователей, ушедших в отток**:
- большинство (90%) имеют кредитный скоринг более 770
- в оттоке в основном - женщины за 58 лет, проживающие в Германии
- в среднем были клиентами банка в течение 4.8 лет
- средний баланс на счету клиента - 91 тыс у.е., предполагаемая зп - 101 тыс у.е.
- большинство имеют около трех банковских продуктов

In [ ]:
df.query("exited == 0").describe(percentiles=[.50, .9], include = 'all').round(2).fillna('-')

**Портрет оставшихся пользователей**:
- в большинстве случаев - французы до 50 лет с кредитным скорингом -- порядка 778
- в среднем являются клиентами банка около 5 лет
- средний баланс на счету клиента - 73 тыс у.е., предполагаемая зп - 100 тыс у.е.
- большинство имеют около двух банковских продуктов

Оба типа клиента (по факту оттока) почти не отличаются количеством лет `tenure`, а также фактом наличия кредитной карты. Это также подтвеждается коррелацией признаков.

## Подготовка данных

Перед созданием модели необходимо проверсти подготовку данных:
- кодирование категориальных переменных;
- разбиение на выборки;
- нормирование числовых данных.
    
Как увидели выше, в датасете есть категориальные признаки (география клиента и его пол), которые необходимо преобразовать в численные. Например, методом прямого кодирования `OneHotEncoding`.

In [ ]:
data_ohe = pd.get_dummies(df, columns=['geography', 'gender'], drop_first=True)
data_ohe.sample(5)

In [ ]:
for column in data_ohe.columns:
    data_ohe[column] = data_ohe[column].astype('int8')

Зафиксируем псевдослучайность для всех используемых в проекте алгоритмов:

In [ ]:
rnd_st = 12345

Напишем функции для дальнейшего использования:
* **target_features** : выделим целевой признак `target` и признаки для обучения модели `features`
* **split_data** : разобьем выборку с помощью метода `train_test_split` в пропорции 60\20\20 на 
    - обучающую *_train
    - валидационную *_valid
    - тестовую выборки *_test

 и выполним масштабирование количественных признаков (скаллер обучаем на обучающей выборке и применяем ко всем)

In [ ]:
def target_features(data_ohe, target_column='exited'):
    '''Функция возвращает целевой признак и признаки для обучения модели'''    
    target = data_ohe[target_column]
    features = data_ohe.drop(target_column, axis=1)
    return target, features

In [ ]:
def split_data(features, target):
    '''Функция разделения выборки'''
    features_train, features_valid, target_train, target_valid = train_test_split(
        features, target, train_size=0.6, test_size=0.4, random_state=rnd_st)

    features_test, features_valid, target_test, target_valid = train_test_split(
        features_valid, target_valid, test_size=0.5, random_state=rnd_st)
    
    scaler = StandardScaler()
    numeric = ['credit_score', 'age', 'balance', 'estimated_salary', 'tenure', 'num_of_products']
    scaler.fit(features_train[numeric])
    
    features_train[numeric] = scaler.transform(features_train[numeric])
    features_valid[numeric] = scaler.transform(features_valid[numeric])
    features_test[numeric] = scaler.transform(features_test[numeric])
    
    return features_train, features_valid, features_test, target_train, target_valid, target_test

Данные готовы к исследованию.

## Исследование задачи

Посмотрим, в каком соотношении встречается целевой признак.

In [ ]:
data_ohe['exited'].value_counts(normalize=True)

Видим, что соотношение классов далеко от 1:1, т.е. они несбалансированы: ~80% отрицательных и ~20% положительных.

При бисбалансе классов применим несколько моделий обучения и посмотрим, какая из них покажет лучшие результаты метрик f1 и roc_auc. Для этого напишем функцию `check_model`.

In [ ]:
def check_model(model, test = None, samplefunc = None, threshold = None):
    '''Функция возвращает модель и ее показатели:
       - выделяет целевой признак и признаки модели
       - делит выборку на обучающую, валидационную, тестовую
       - применяет к выборкам функцию увеличения, если необходимо
       - использует валидационную выборку, если не указана тестовая
       - выбирает лучший порог
       - показывает метрики f1 и auc модели 
    '''
    
    target, features = target_features(data_ohe)
    
    features_train, features_valid, features_test, target_train, target_valid, target_test = split_data(features, target)
    
    if samplefunc:
        features_train, target_train = samplefunc(features_train, target_train)
        
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    f1 = f1_score(target_valid, predicted_valid)
    roc_auc = roc_auc_score(target_valid, predicted_valid, multi_class='ovr')
    
    if test:
        predicted_test = model.predict(features_test)
        f1 = f1_score(target_test, predicted_test)
        roc_auc = roc_auc_score(target_test, predicted_test, multi_class='ovr')
        
    if threshold:
        probabilities_valid = model.predict_proba(features_valid)
        probabilities_one_valid = probabilities_valid[:, 1]
        best_th = 0
        best_f1 = 0
        for threshold in np.arange(0, 0.7, 0.05):
            predicted_valid = probabilities_one_valid > threshold
            predicted = model.predict(features) 
            f1 = f1_score(target_valid, predicted_valid)
            roc_auc = roc_auc_score(target_valid, predicted_valid, multi_class='ovr')
            if f1 > best_f1:
                best_f1 = f1
                best_th = threshold
        print(' | Лучший порог= {:.2f}'.format(threshold), end='')        
            
    print(" | f1 = {:.5f} | roc_auc = {:.5f}".format(f1, roc_auc))

### Дерево решений

In [ ]:
for max_depth in range(2, 22, 2):
    print("max_depth = ", max_depth, end='')
    model = check_model(DecisionTreeClassifier(max_depth=max_depth, random_state=rnd_st))    

Пока лучшие результаты f1=0.52067 получили у модели с глубиной равной 6.

### Случайный лес

В качестве гиперпараметра глубины дерева возьмем значение, найденное для предыдущей модели. 
А количество деревьев для нашего случайного леса будет искать в диапазоне от 10 до 100 с шагом 10.

In [ ]:
for estim in range(10, 101, 10):
    print("estim = ", estim, end='')
    model = check_model(RandomForestClassifier(n_estimators=estim, max_depth=6, random_state=rnd_st))

Как видим, модель случайного леса предсказывает не лучше, чем дерево решений. Лучший результат 0.47315 достигается при количестве деревьев равном 60.

### Логистическая регрессия

In [ ]:
check_model(LogisticRegression(random_state = rnd_st))

При дисбалансе классов логистическая регрессия предсказывает крайне плохо.

Самая лучшая модель -- дерево решений, с учетом дисбаланс классов должна показать хорошие результаты.

## Борьба с дисбалансом

### Взвешивание классов

Придадим объектам редкого класса 1 больший вес используя гиперпараметр class_weight='balanced'.

#### Дерево решений

In [ ]:
for max_depth in range(2, 22, 2):
    print("max_depth = ", max_depth, end='')
    model = check_model(DecisionTreeClassifier(class_weight='balanced', max_depth=max_depth, random_state=rnd_st))  

Лучшая модель с глубиной дерева -- 8 (f1 = 0.57576).

#### Случайный лес

In [ ]:
for est in range(10, 101, 10):
    print("est = ", est, end='')
    model = check_model(RandomForestClassifier(class_weight='balanced', 
                                               n_estimators=est, 
                                               max_depth=8, 
                                               random_state=rnd_st))

А вот модель случайного леса показывает самое высокое значение F1-меры: 0.59251 при количестве деревьев равном 100.

#### Логистическая регрессия

In [ ]:
check_model(LogisticRegression(class_weight='balanced', random_state = rnd_st))

Особых улучшений нет.

### Увеличение выборки
Cделаем объекты редкого класса не такими редкими и переобучим модели.

Попробуем увеличить `upsample` выборку для увеличения метрик. Для этого напишем соответствующую функцию и попробуем подобрать лучшее значение увеличения выборки с помощью функции `map`.

In [ ]:
def upsampled(features, target, repeat=10):
    ''' Функция увеличения выборки для увеличения метрик качества'''    
    features_0 = features[target == 0]
    features_1 = features[target == 1]
    target_0 = target[target == 0]
    target_1 = target[target == 1]
    
    # увеличение
    features_up = pd.concat([features_0] + [features_1] * repeat)
    target_up = pd.concat([target_0] + [target_1] * repeat)
    
    # перемешиваем
    features_up, target_up = shuffle(features_up, target_up, random_state=rnd_st)  
    
    return features_up, target_up

#### Дерево решений

In [ ]:
for max_depth in range(2, 22, 2):
    print("max_depth = ", max_depth, end='')
    model = check_model(DecisionTreeClassifier(max_depth=max_depth, random_state=rnd_st), 
                        samplefunc = upsampled) 

Самый лучший результат у модели с 12 деревьями (f1 = 0.51459).

#### Случайный лес

In [ ]:
for est in range(10, 101, 10):
    print("est = ", est, end='')
    model = check_model(RandomForestClassifier(n_estimators=est, 
                                               max_depth=12, 
                                               random_state=rnd_st), samplefunc = upsampled)

Самый лучший результат у модели с глубиной 40 -- f1 = 0.58870.

#### Логистическая регрессия

In [ ]:
check_model(LogisticRegression(random_state = rnd_st), samplefunc = upsampled)

### Увеличение порога

Посмотрим лучший порог на каждой модели

#### Дерево решений

In [ ]:
for max_depth in range(2, 22, 2):
    print("max_depth = ", max_depth, end='')
    model = check_model(DecisionTreeClassifier(max_depth=max_depth, random_state=rnd_st), 
                        threshold = 1) 

#### Случайный лес

In [ ]:
for est in range(10, 101, 10):
    print("est = ", est, end='')
    model = check_model(RandomForestClassifier(n_estimators=est, 
                                               max_depth=12, 
                                               random_state=rnd_st), threshold = 1)

#### Логистическая регрессия

In [ ]:
check_model(LogisticRegression(random_state = rnd_st), threshold = 1)

Лучший уровень порога для лубой модели -- 0.65, при этом метрика f1 не дает результатов лучше, яем другие методы борьбы с дисбалансом.

**Как итог, лучший примененный метод -- взвешивание классов, а лучшая модель -- случайный лес с глубиной 100.**

## Тестирование модели

Проверим самую лучшую модель по итогам прошлого пункта (`случайный лес`) с количеством деревьев равным 1000 и глубиной равной 8 с учетом балансировки классов.

In [ ]:
check_model(RandomForestClassifier(class_weight='balanced', n_estimators=100, max_depth=8, random_state=rnd_st))

Протестируем модель на тестовой выборке:

In [ ]:
check_model(RandomForestClassifier(class_weight='balanced', n_estimators=100, max_depth=8, random_state=rnd_st), test = 1)

Достигли желаемого результата - модель предсказывает достаточно хорошо (f1 = 0.62177).

В итоге мы добились неплохих результатов: 
- исследовали баланс классов,
- учли их дисбаланс,
- получили подтвержение качества модели, доказав это расчетом нужных метрик.